<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/sampleSumContracts01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow --quiet
!pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
!pip install "git+https://github.com/ludwig-ai/ludwig.git@master#egg=ludwig[llm]" --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
DEPRECATION: git+https://github.com/ludwig-ai/ludwig.git@master#egg=ludwig[llm] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml
import pandas as pd

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [3]:
summarize_data = [
    {"input": "2015: 3.5M,2016: 3.6M,2017: 3.7M,2018: 3.8M,2019: 3.9M", "instruction": "Summarize the total fees", "output":"18.5M"},
    {"input": "2015: 5.5M,2016: 5.6M,2017: 5.7M,2018: 5.8M,2019: 5.9M", "instruction": "Summarize the total fees", "output":"28.5M"},
    {"input": "2015: 7.5M,2016: 5.6M,2017: 6.7M,2018: 5.8M,2019: 12.9M", "instruction": "Summarize the total fees", "output":"38.5M"},
    {"input": "2015: 10.5M,2016: 3.6M,2017: 5.7M,2018: 5.8M,2019: 5.9M", "instruction": "Summarize the total fees", "output":"31.5M"},
    {"input": "2015: 3.5M,2016: 3.6M,2017: 3.7M,2018: 3.8M,2019: 3.9M", "instruction": "Summarize the total fees", "output":"18.5M"},
    {"input": "2015: 5.5M,2016: 5.6M,2017: 5.7M,2018: 5.8M,2019: 5.9M", "instruction": "Summarize the total fees", "output":"28.5M"},
    {"input": "2015: 7.5M,2016: 5.6M,2017: 6.7M,2018: 5.8M,2019: 12.9M", "instruction": "Summarize the total fees", "output":"38.5M"},
    {"input": "2015: 10.5M,2016: 3.6M,2017: 5.7M,2018: 5.8M,2019: 5.9M", "instruction": "Summarize the total fees", "output":"31.5M"},
]

df = pd.DataFrame(summarize_data)

df.head()

,input,instruction,output
0,"2015: 3.5M,2016: 3.6M,2017: 3.7M,2018: 3.8M,20...",Summarize the total fees,18.5M
1,"2015: 5.5M,2016: 5.6M,2017: 5.7M,2018: 5.8M,20...",Summarize the total fees,28.5M
2,"2015: 7.5M,2016: 5.6M,2017: 6.7M,2018: 5.8M,20...",Summarize the total fees,38.5M
3,"2015: 10.5M,2016: 3.6M,2017: 5.7M,2018: 5.8M,2...",Summarize the total fees,31.5M
4,"2015: 3.5M,2016: 3.6M,2017: 3.7M,2018: 3.8M,20...",Summarize the total fees,18.5M


In [4]:
qlora_fine_tuning_config = yaml.safe_load(
"""
model_type: llm
base_model: huggyllama/llama-7b

input_features:
  - name: instruction
    type: text

output_features:
  - name: output
    type: text

prompt:
  template: >-
    Below is an instruction that describes a task, paired with an input
    that provides further context. Write a response that appropriately
    completes the request.

    ### Instruction: {instruction}

    ### Input: {input}

    ### Response:

generation:
  temperature: 0.1
  max_new_tokens: 100

adapter:
  type: lora

quantization:
  bits: 4

trainer:
  type: finetune
  epochs: 5
  batch_size: 1
  eval_batch_size: 2
  gradient_accumulation_steps: 16
  learning_rate: 0.00001
  optimizer:
    type: adam
    params:
      eps: 1.e-8
      betas:
        - 0.9
        - 0.999
      weight_decay: 0
  learning_rate_scheduler:
    decay: cosine
    warmup_fraction: 0.03
    reduce_on_plateau: 0
"""
)

model = LudwigModel(config=qlora_fine_tuning_config, logging_level=logging.INFO)
results = model.train(dataset=df)

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run_2                                                   │
├──────────────────┼─────────────────────────────────────────────────────────────────

OSError: ignored

In [ ]:
test_data_pred = summarize_data = [
    {"input": "2015: 2.5M,2016: 8.6M,2017: 3.7M,2018: 3.8M,2019: 4.9M", "instruction": "Summarize the total fees"},
    {"input": "2015: 5.5M,2016: 5.6M,2017: 1.7M,2018: 2.8M,2019: 11.9M", "instruction": "Summarize the total fees"},
    {"input": "2015: 6.5M,2016: 5.6M,2017: 3.7M,2018: 5.8M,2019: 12.9M", "instruction": "Summarize the total fees"}]

df2 = pd.DataFrame(test_data_pred)
df2.head(10)

In [ ]:
predictions = model.predict(df)[0]
for input_with_prediction in zip(df['instruction'], df['input'], predictions['output_response']):
  print(f"Instruction: {input_with_prediction[0]}")
  print(f"Input: {input_with_prediction[1]}")
  print(f"Generated Output: {input_with_prediction[2][0]}")
  print("\n\n")